In [70]:
import numpy as np
import pandas as pd

# train_data = pd.read_csv('twitter_train.csv')
# test_data = pd.read_csv('twitter_test.csv')
total_data = pd.read_csv('Total_Tweets.csv')

# test_data = pd.read_csv('custom_test.csv')
# print(test_data)

In [68]:
train_data.shape

(11712, 15)

In [69]:
test_data.shape

(2928, 15)

In [78]:
## train_docs
import random
# Test Train Split
LEN = len(total_data)
TRAIN_LEN = int((0.8)*LEN)
train_data=total_data[:TRAIN_LEN]
test_data=total_data[TRAIN_LEN:]

train_docs = []
for i in range(train_data.shape[0]):
    train_docs.append((train_data['text'][i], train_data['airline_sentiment'][i]))
random.shuffle(train_docs)

#train_docs


In [70]:
import numpy as np
import pandas as pd

# train_data = pd.read_csv('twitter_train.csv')
# test_data = pd.read_csv('twitter_test.csv')
total_data = pd.read_csv('Total_Tweets.csv')

# test_data = pd.read_csv('custom_test.csv')
# print(test_data)

In [16]:
# test_data

In [77]:
#test_docs
test_docs = []
# for i in range(test_data.shape[0]):
#     test_docs.append((test_data['text'][i], test_data['airline_sentiment_gold'][i]))
for index, row in test_data.iterrows():
    test_docs.append((row['text'], row['airline_sentiment_gold']))

In [79]:
new_train_docs = [(doc.split(' '), cat) for doc, cat in train_docs]
new_test_docs = [(doc.split(' '), cat) for doc, cat in test_docs]

In [59]:
#  new_train_docs[0]

In [80]:
## stop words:
import nltk
from nltk.corpus import stopwords
# nltk.download('stopwords')
import string
stop = stopwords.words('english')
punctuations = list(string.punctuation)
a = ['']
stop = stop + punctuations + a

In [61]:
#stop

In [81]:
###lemmatize each word and find new claen_docs for training :

from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
lemmatizer = WordNetLemmatizer()

In [82]:
## defining a func for simple_pos_tag
from nltk.corpus import wordnet
def get_simple_pos(tag):
    
    if tag.startswith('J'):
        return wordnet.ADJ
    
    elif tag.startswith('V'):
        return wordnet.VERB
    
    elif tag.startswith('N'):
        return wordnet.NOUN
    
    elif tag.startswith('R'):
        return wordnet.ADV
    
    else:
        return wordnet.NOUN

In [84]:
def clean_analysis(words):
    output_words = []
    for w in words:
        if w.lower() not in stop:
            pos = pos_tag([w])
            clean_words = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_words.lower())
    return output_words

In [85]:
### get clean train and test docs
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
clean_train_docs = [(clean_analysis(doc),cat) for doc, cat in new_train_docs]
clean_test_docs = [(clean_analysis(doc),cat) for doc, cat in new_test_docs]


In [66]:
# clean_train_docs[0]

In [86]:
from sklearn.feature_extraction.text import CountVectorizer

In [100]:
sentiments = [cat for doc, cat in clean_train_docs]
train_tweets = [' '.join(doc) for doc, cat in clean_train_docs]
test_tweets = [' '.join(doc) for doc, cat in clean_test_docs]

11712


In [88]:
count_vec = CountVectorizer(max_features = 3000, max_df = 0.98, ngram_range = (1,3))

In [89]:
x_train_features = count_vec.fit_transform(train_tweets)

In [90]:
x_train_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [91]:
#count_vec.get_feature_names()

In [35]:
x_test_features = count_vec.transform(test_tweets)

In [103]:
x_test_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [102]:
# count_vec.get_feature_names()
# print(sentiments)

In [105]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
clf=SVC()
grid={'C':[1e2,1e4,1e1,5e3,1e3,5e2],
      'gamma':[1e-3,1e-2,5e-3,5e-2,1e-4,1e-5]}
svm=GridSearchCV(clf,grid,n_jobs=-1)
svm.fit(x_train_features, sentiments)


GridSearchCV(estimator=SVC(), n_jobs=-1,
             param_grid={'C': [100.0, 10000.0, 10.0, 5000.0, 1000.0, 500.0],
                         'gamma': [0.001, 0.01, 0.005, 0.05, 0.0001, 1e-05]})

In [106]:
print(svm.best_score_)
print(svm.best_params_)
y_pred = svm.predict(x_test_features)
#print(x_test_features)

0.7691262342577578
{'C': 10.0, 'gamma': 0.01}


In [107]:
predictions = pd.DataFrame(y_pred)

In [108]:
predictions.to_csv('twitter_analysis.csv', index = False, header = False)